In [ ]:
!pip install keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [ ]:
train_images = train_images/255.0
test_images = test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):
  model = keras.Sequential([
keras.layers.Conv2D(
filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
activation='relu',
input_shape=(28,28,1)
  ),
keras.layers.Conv2D(
filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
activation='relu'
  ),
keras.layers.Flatten(),
keras.layers.Dense(
units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
activation='relu'
  ),
keras.layers.Dense(10, activation='softmax')#output layer
  ])
 
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
loss='sparse_categorical_crossentropy',
metrics=['accuracy'])
  return model             


In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name='Mnist Fashion')

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 2 Complete [00h 23m 23s]
val_accuracy: 0.9078333377838135

Best val_accuracy So Far: 0.9096666574478149
Total elapsed time: 00h 35m 46s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |48                |32                
conv_1_kernel     |3                 |3                 
conv_2_filter     |64                |48                
conv_2_kernel     |3                 |5                 
dense_1_units     |64                |80                
learning_rate     |0.001             |0.001             

Epoch 1/3
 782/1688 [============>.................] - ETA: 1:52 - loss: 0.4621 - accuracy: 0.8375

In [ ]:
model=tuner_search.get_best_model(num_model=1)[0]

In [ ]:
model.summary